# Music Genre Classification - ECS 171 Final Project

In [1]:
import kagglehub
import os
import pandas as pd

In [2]:
path = kagglehub.dataset_download("vicsuperman/prediction-of-music-genre")
print("Files in dataset folder:", os.listdir(path))
df = pd.read_csv(os.path.join(path, "music_genre.csv"))
df.head()

100%|██████████| 2.72M/2.72M [00:00<00:00, 5.71MB/s]

Extracting files...
Files in dataset folder: ['music_genre.csv']


,instance_id,artist_name,track_name,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,obtained_date,valence,music_genre
0,32894.0,Röyksopp,Röyksopp's Night Out,27.0,0.00468,0.652,-1.0,0.941,0.79200,A#,0.115,-5.201,Minor,0.0748,100.889,4-Apr,0.759,Electronic
1,46652.0,Thievery Corporation,The Shining Path,31.0,0.01270,0.622,218293.0,0.890,0.95000,D,0.124,-7.043,Minor,0.0300,115.00200000000001,4-Apr,0.531,Electronic
2,30097.0,Dillon Francis,Hurricane,28.0,0.00306,0.620,215613.0,0.755,0.01180,G#,0.534,-4.617,Major,0.0345,127.994,4-Apr,0.333,Electronic
3,62177.0,Dubloadz,Nitro,34.0,0.02540,0.774,166875.0,0.700,0.00253,C#,0.157,-4.498,Major,0.2390,128.014,4-Apr,0.270,Electronic
4,24907.0,What So Not,Divide & Conquer,32.0,0.00465,0.638,222369.0,0.587,0.90900,F#,0.157,-6.266,Major,0.0413,145.036,4-Apr,0.323,Electronic


## 1. Setup & Imports



# import libraries, set seed, configure plotting

# read CSV, drop cols, encode, standardize


# histograms, heatmap, genre distribution, boxplots


# pick top features, visualize, maybe PCA


# train 3 models, grid search, store best params


# accuracy, f1, confusion matrix, ROC


In [5]:
import numpy as np
import tensorflow
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import label_binarize
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import SGDClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.optimizers import SGD

# clean the dataset
df.drop(['instance_id', 'artist_name', 'track_name', 'obtained_date'], axis=1, inplace=True)
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

# select X and y
X = df.drop('music_genre', axis=1)
y = df['music_genre']

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Standardize the features of the training adn tesitng sets
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Train SGDclassifier
model = SGDClassifier(loss='squared', penalty = 'l2', alpha = 0.0001, max_iter = 100, eta0 = 0.01, random_state=42)
model.fit(X_train, y_train)



# Create Neural Network Architecture
def NNmodel_struct(num_layers = 6, learning_rate = 0.001, dropout_rate = 0.2):
    model = Sequential()
    model.add(Dense(13, activation='relu', input_shape = (X.shape[1],))) # input layer, 13 nodes for 13 features
    model.add(Dropout(dropout_rate))
    num_nodes_init_layer = 512
    for i in range(num_layers - 1):
        model.add(Dense(num_nodes_init_layer, activation = 'relu'))
        model.add(Dropout(dropout_rate))
        num_nodes_init_layer = num_nodes_init_layer // 2
    model.add(Dense(1, activation = 'sigmoid'))

    optimizer = SGD()
    model.compile(optimizer=optimizer, loss = 'mse', metrics = ['accuracy'])
    return model

param_grid = {
    'num_layers': [6, 7, 8],
    'dropout_rate': [0.2, 0.25, 0.3],
    'learning_rate': [0.001, 0.01]
}


# create and train grid object
NNmodel = KerasClassifier(build_fn=NNmodel_struct, epochs = 50, batch_size=32, verbose=0)
grid = GridSearchCV(estimator=NNmodel, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train)



# Train SGDclassifier
model = SGDClassifier(loss='squared', penalty = 'l2', alpha = 0.0001, max_iter = 100, eta0 = 0.01, random_state=42)
model.fit(X_train, y_train)

# Predict labels and probabilities
y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)

# Encode string labels
label_encoder = LabelEncoder() 
y_test_encoded = label_encoder.fit_transform(y_test)

# Binarize encoded labels
classes = list(range(len(label_encoder.classes_))) 
y_test_bin = label_binarize(y_test_encoded, classes=classes)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy:{accuracy:.4f}")

# Calculate f1 (assumes uneven distribution of genres)
f1 = f1_score(y_test, y_pred, average='weighted')
print(f"f1 score:{f1:.4f}")

# Calculate precision
precision = precision_score(y_test, y_pred, average='weighted')
print(f"Precision: {precision:.4f}")

# Calculate f1
recall = recall_score(y_test, y_pred, average='weighted')
print(f"Recall: {recall:.4f}")

# Calculate ROC AUC
roc_auc = roc_auc_score(y_test_bin, y_prob, average='weighted', multi_class='ovr')
print(f"ROC AUC Score: {roc_auc:.4f}")

# Calculate confusion matrix
confusion_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", confusion_matrix)

KeyError: "['instance_id', 'artist_name', 'track_name', 'obtained_date'] not found in axis"

# analysis, takeaway, which model wins
